# OMERO Annotation Workflow from YAML Configuration

Load an existing YAML configuration and run the annotation pipeline.

In [ ]:
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    load_config_from_yaml,
    create_pipeline,
)
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")

## 1. OMERO Connection

In [ ]:
conn_widget = create_omero_connection_widget()
conn_widget.display()

In [ ]:
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established.")

print(f"Connected as: {conn.getUser().getName()}")

## 2. Load Configuration from YAML

Provide the path to your YAML configuration file.

In [ ]:
# Update the path to your YAML configuration file
config_path = "path/to/your/annotation_config.yaml"

config = load_config_from_yaml(config_path)

print("Configuration Summary:")
print(f"  Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"  Workflow Name: {config.name}")
print(f"  Model: {config.ai_model.model_type}")
print(f"  Channel: {config.spatial_coverage.channels}")
print(f"  Output: {config.output.output_directory}")
print(f"  Read-only Mode: {config.workflow.read_only_mode}")

## 3. Run the Pipeline

In [ ]:
pipeline = create_pipeline(config, conn)

# Run the complete micro-SAM workflow
table_id, updated_config = pipeline.run_full_micro_sam_workflow()

print(f"Annotation complete. Table ID: {table_id}")
print(f"Images processed: {len(updated_config.get_processed())}")

## 4. Save Updated Configuration

In [ ]:
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"Configuration saved to: {config_path}")
except Exception as e:
    config.save_yaml(config_filename)
    print(f"Configuration saved to: {config_filename}")

## 5. Cleanup

In [ ]:
if "conn" in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed.")

## Usage Options

